### XGBoost cloud training template for Diabetes Prediction

In [1]:
import numpy as np
import pandas as pd

import boto3
import re
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Import s3 bucket name from .env file

import os
env_vars = !cat ./.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key]=value

In [3]:
bucket_name = os.environ['BUCKET_NAME']

In [ ]:
bucket_name

In [5]:
# bucket name and folders for Diabetes files in s3

training_file_key = 'diabetes/diabetes_train.csv'
validation_file_key = 'diabetes/diabetes_validation.csv'

s3_model_output_location = r's3://{0}/iris/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name, training_file_key)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name, validation_file_key)

In [ ]:
print(s3_model_output_location)
print(s3_training_file_location)
print(s3_validation_file_location)

In [7]:
# function to write files into s3

def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f:
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [8]:
write_to_s3('diabetes_train_imputed.csv',bucket_name,training_file_key)
write_to_s3('diabetes_validation_imputed.csv',bucket_name, validation_file_key)

In [9]:
diabetes_training_column_list_file_key = 'diabetes/diabetes_training_column_list.txt'
s3_training_column_list_file_location = r's3://{0}/{1}'.format(bucket_name, diabetes_training_column_list_file_key)

In [10]:
write_to_s3('diabetes_training_column_list.txt',bucket_name, diabetes_training_column_list_file_key)

### Training XGBoost Algorithm in Docker Image

#### using the AWS maintained xgboost image for every region

In [13]:
# Use spot instance

use_spot_instance = True
max_run = 3600
max_wait = 7200

job_name = 'xgboost-diabetes-v1'

checkpoint_s3_uri = None

if use_spot_instance:
    checkpoint_s3_uri = f's3://{bucket_name}/diabetes/checkpoints/{job_name}'

In [14]:
sess = sagemaker.Session()

In [15]:
role = get_execution_role()

In [ ]:
print(role)

In [17]:
container = sagemaker.image_uris.retrieve("xgboost",sess.boto_region_name, version='1.2-2')
print(f'Using XGBoost container:{container}')

Using XGBoost container:683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2


### Build Model

In [67]:
# Configure the training job
# Specify type and number of instances will be used
# s3 location where final artifacts needs to be stored
#   Reference: http://sagemaker.readthedocs.io/en/latest/estimators.html

# for managed spot training, specify the use_spot_instances flag, max_run, max_wait and checkpoint_s3_uri

# SDK 2.x version does not require train prefix for instance count and type

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path = s3_model_output_location,
    sagemaker_session = sess,
    base_job_name=job_name,
    use_spot_instances=use_spot_instance,
    max_run=max_run,
    max_wait=max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri
)

In [69]:
# Specify hyper parameters that appropriate for the training algorithm
# XGBoost Training parameter Reference:

estimator.set_hyperparameters(
    max_depth=5,
    objective='binary:logistic',
    eval_metric='logloss',
    num_round=100,
    early_stopping_rounds=10
)

In [70]:
estimator.hyperparameters()

{'max_depth': 5,
 'objective': 'binary:logistic',
 'eval_metric': 'logloss',
 'num_round': 100,
 'early_stopping_rounds': 10}

In [71]:
### Preparing Training data locations

training_input_config = sagemaker.session.TrainingInput(
    s3_data = s3_training_file_location,
    content_type = 'csv',
    s3_data_type = 'S3Prefix'
)

validation_input_config = sagemaker.session.TrainingInput(
    s3_data = s3_validation_file_location,
    content_type = 'csv',
    s3_data_type = 'S3Prefix'
)

data_channels = {'train':training_input_config, 'validation':validation_input_config}

In [72]:
print(training_input_config)
print(validation_input_config)

In [73]:
print(data_channels)

{'train': <sagemaker.inputs.TrainingInput object at 0x7f720d9ec550>, 'validation': <sagemaker.inputs.TrainingInput object at 0x7f720d9ef970>}


### Train the model

In [74]:
# XGBoost supports "train", "validation" data channels

estimator.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-diabetes-v1-2024-05-04-22-03-06-826


2024-05-04 22:03:06 Starting - Starting the training job...
2024-05-04 22:03:20 Starting - Preparing the instances for training...
2024-05-04 22:03:57 Downloading - Downloading input data...
2024-05-04 22:04:23 Downloading - Downloading the training image...
2024-05-04 22:04:58 Training - Training image download completed. Training in progress..[2024-05-04 22:05:07.978 ip-10-2-115-53.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-05-04:22:05:08:INFO] Imported framework sagemaker_xgboost_container.training
[2024-05-04:22:05:08:INFO] Failed to parse hyperparameter eval_metric value logloss to Json.
Returning the value itself
[2024-05-04:22:05:08:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-05-04:22:05:08:INFO] No GPUs detected (normal if no gpus installed)
[2024-05-04:22:05:08:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-05-04:22:05:08:INFO] Determined delimiter of CSV input is ','
[2

### Deploy Predictor Model

In [76]:
predictor = estimator.deploy(
                        initial_instance_count=1,
                        instance_type='ml.m5.xlarge',
                        endpoint_name=job_name)

INFO:sagemaker:Creating model with name: xgboost-diabetes-v1-2024-05-04-22-07-12-539
INFO:sagemaker:Creating endpoint-config with name xgboost-diabetes-v1
INFO:sagemaker:Creating endpoint with name xgboost-diabetes-v1


-----!

### Try Predicting

In [77]:
# SDK 2.0 serializers

from sagemaker.serializers import CSVSerializer
predictor.serializer = CSVSerializer()

In [78]:
predictor.predict(
    [[1.0,130.0,70.0,13.0,105.0,25.9,0.472,22],
[8.0,133.0,72.0,33.0,207.0,32.9,0.27,39],
[9.0,130.0,70.0,33.0,207.0,34.2,0.652,45],
[7.0,107.0,74.0,33.0,207.0,29.6,0.254,31]]
)

b'0.22223027050495148\n0.9175506830215454\n0.9175506830215454\n0.9175506830215454\n'